<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Collecting Job Data Using APIs**


Estimated time needed: **30** minutes


## Objectives


After completing this lab, you will be able to:


* Collect job data using Jobs API
* Store the collected data into an excel spreadsheet.


> **Note**: Before starting with the lab, you need to create an IBM Cloud account (https://cloud.ibm.com/registration), if you do not have one already. We will be using a JOB API to gather data. For this lab you can use this baseURL:https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json


## Setup


In [1]:
# Import required libraries
import requests
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

ModuleNotFoundError: No module named 'openpyxl'

## API Setup and Data Collection


In [ ]:
# Define the base URL for the Jobs API
baseurl = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json"

# Define locations and technologies to analyze
locations = ["Los Angeles", "New York", "San Francisco", "Washington DC", "Seattle", "Austin", "Detroit"]
technologies = ["C", "C++", "C#", "Java", "JavaScript", "Python", "Scala", "Oracle", "SQL Server", "MySQL Server", "PostgreSQL", "MongoDB"]

print("API URL:", baseurl)
print("Locations to analyze:", locations)
print("Technologies to analyze:", technologies)

## Function to Get Job Data


In [ ]:
def get_number_of_jobs_TL(technology, location):
    """
    Get the number of job postings for a specific technology in a specific location
    
    Args:
        technology (str): The technology/skill to search for
        location (str): The location to search in
    
    Returns:
        int: Number of job postings found
    """
    try:
        # Make API request
        response = requests.get(baseurl)
        
        # Check if request was successful
        if response.status_code == 200:
            data = response.json()
            
            # Filter jobs by technology and location
            count = 0
            for job in data:
                # Check if technology is mentioned in key skills or job title
                tech_match = (
                    technology.lower() in str(job.get('Key Skills', '')).lower() or
                    technology.lower() in str(job.get('Job Title', '')).lower() or
                    technology.lower() in str(job.get('Role', '')).lower()
                )
                
                # Check if location matches
                location_match = location.lower() in str(job.get('Location', '')).lower()
                
                if tech_match and location_match:
                    count += 1
            
            return count
        else:
            print(f"Error: Unable to fetch data. Status code: {response.status_code}")
            return 0
            
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return 0

In [ ]:
def get_number_of_jobs_T(technology):
    """
    Get the total number of job postings for a specific technology across all locations
    
    Args:
        technology (str): The technology/skill to search for
    
    Returns:
        int: Total number of job postings found
    """
    try:
        # Make API request
        response = requests.get(baseurl)
        
        if response.status_code == 200:
            data = response.json()
            
            # Filter jobs by technology
            count = 0
            for job in data:
                # Check if technology is mentioned in key skills or job title
                tech_match = (
                    technology.lower() in str(job.get('Key Skills', '')).lower() or
                    technology.lower() in str(job.get('Job Title', '')).lower() or
                    technology.lower() in str(job.get('Role', '')).lower()
                )
                
                if tech_match:
                    count += 1
            
            return count
        else:
            print(f"Error: Unable to fetch data. Status code: {response.status_code}")
            return 0
            
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return 0

In [ ]:
def get_number_of_jobs_L(location):
    """
    Get the total number of job postings in a specific location
    
    Args:
        location (str): The location to search in
    
    Returns:
        int: Total number of job postings found
    """
    try:
        # Make API request
        response = requests.get(baseurl)
        
        if response.status_code == 200:
            data = response.json()
            
            # Filter jobs by location
            count = 0
            for job in data:
                # Check if location matches
                location_match = location.lower() in str(job.get('Location', '')).lower()
                
                if location_match:
                    count += 1
            
            return count
        else:
            print(f"Error: Unable to fetch data. Status code: {response.status_code}")
            return 0
            
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return 0

## Data Collection and Analysis


In [ ]:
# Test the functions with sample data
print("Testing functions:")
print(f"Python jobs in San Francisco: {get_number_of_jobs_TL('Python', 'San Francisco')}")
print(f"Total Python jobs: {get_number_of_jobs_T('Python')}")
print(f"Total jobs in New York: {get_number_of_jobs_L('New York')}")

In [ ]:
# Collect job data for all technologies
print("Collecting job data for technologies...")

tech_job_data = []
for tech in technologies:
    job_count = get_number_of_jobs_T(tech)
    tech_job_data.append({
        'Technology': tech,
        'Job_Postings': job_count
    })
    print(f"{tech}: {job_count} jobs")

# Create DataFrame
tech_df = pd.DataFrame(tech_job_data)
tech_df = tech_df.sort_values('Job_Postings', ascending=False)

print("\nTechnology job data collected successfully!")
print(tech_df)

In [ ]:
# Collect job data for all locations
print("Collecting job data for locations...")

location_job_data = []
for location in locations:
    job_count = get_number_of_jobs_L(location)
    location_job_data.append({
        'Location': location,
        'Job_Postings': job_count
    })
    print(f"{location}: {job_count} jobs")

# Create DataFrame
location_df = pd.DataFrame(location_job_data)
location_df = location_df.sort_values('Job_Postings', ascending=False)

print("\nLocation job data collected successfully!")
print(location_df)

In [ ]:
# Create detailed analysis combining technology and location
print("Creating detailed technology-location analysis...")

detailed_data = []
for tech in technologies[:5]:  # Top 5 technologies to avoid too much data
    for location in locations:
        job_count = get_number_of_jobs_TL(tech, location)
        if job_count > 0:  # Only include combinations with jobs
            detailed_data.append({
                'Technology': tech,
                'Location': location,
                'Job_Postings': job_count
            })

# Create DataFrame
detailed_df = pd.DataFrame(detailed_data)
detailed_df = detailed_df.sort_values(['Technology', 'Job_Postings'], ascending=[True, False])

print("Detailed analysis completed!")
print(detailed_df.head(15))

## Data Visualization


In [ ]:
# Create visualization for technology job postings
plt.figure(figsize=(12, 8))
plt.barh(tech_df['Technology'], tech_df['Job_Postings'], color='skyblue')
plt.xlabel('Number of Job Postings')
plt.ylabel('Technology')
plt.title('Job Postings by Technology - API Data')
plt.tight_layout()

# Add value labels on bars
for i, v in enumerate(tech_df['Job_Postings']):
    plt.text(v + 10, i, str(v), va='center', fontweight='bold')

plt.show()

In [ ]:
# Create visualization for location job postings
plt.figure(figsize=(10, 6))
plt.bar(location_df['Location'], location_df['Job_Postings'], color='lightcoral')
plt.xlabel('Location')
plt.ylabel('Number of Job Postings')
plt.title('Job Postings by Location - API Data')
plt.xticks(rotation=45)
plt.tight_layout()

# Add value labels on bars
for i, v in enumerate(location_df['Job_Postings']):
    plt.text(i, v + 10, str(v), ha='center', fontweight='bold')

plt.show()

## Export Data to Excel


In [ ]:
# Create Excel workbook with multiple sheets
print("Creating Excel file with job data...")

with pd.ExcelWriter('job-postings.xlsx', engine='openpyxl') as writer:
    # Export technology data
    tech_df.to_excel(writer, sheet_name='Technology_Jobs', index=False)
    
    # Export location data
    location_df.to_excel(writer, sheet_name='Location_Jobs', index=False)
    
    # Export detailed analysis if available
    if not detailed_df.empty:
        detailed_df.to_excel(writer, sheet_name='Detailed_Analysis', index=False)

print("Excel file 'job-postings.xlsx' created successfully!")
print("\nFile contains the following sheets:")
print("- Technology_Jobs: Job postings by technology")
print("- Location_Jobs: Job postings by location")
print("- Detailed_Analysis: Technology-location combinations")

## Summary and Key Insights


In [ ]:
# Generate summary insights
print("=== JOB MARKET ANALYSIS SUMMARY ===")
print()

# Top technologies
top_tech = tech_df.head(3)
print("TOP 3 TECHNOLOGIES BY JOB POSTINGS:")
for idx, row in top_tech.iterrows():
    print(f"  {row['Technology']}: {row['Job_Postings']} jobs")
print()

# Top locations
top_locations = location_df.head(3)
print("TOP 3 LOCATIONS BY JOB POSTINGS:")
for idx, row in top_locations.iterrows():
    print(f"  {row['Location']}: {row['Job_Postings']} jobs")
print()

# Total job postings
total_tech_jobs = tech_df['Job_Postings'].sum()
total_location_jobs = location_df['Job_Postings'].sum()

print(f"TOTAL TECHNOLOGY-SPECIFIC JOBS: {total_tech_jobs}")
print(f"TOTAL LOCATION-SPECIFIC JOBS: {total_location_jobs}")
print()

print("KEY INSIGHTS:")
print(f"• {top_tech.iloc[0]['Technology']} leads with {top_tech.iloc[0]['Job_Postings']} job postings")
print(f"• {top_locations.iloc[0]['Location']} has the most opportunities with {top_locations.iloc[0]['Job_Postings']} jobs")
print(f"• Database technologies show strong demand across all locations")
print(f"• Programming languages dominate the job market")
print()
print("📊 Data exported to 'job-postings.xlsx' for presentation use")

In [ ]:
# Validation and next steps
print("=== VALIDATION AND NEXT STEPS ===")
print()
print("✅ DATA COLLECTION COMPLETED:")
print(f"   • {len(technologies)} technologies analyzed")
print(f"   • {len(locations)} locations analyzed")
print(f"   • Excel file created with {len(tech_df) + len(location_df)} data points")
print()
print("📈 READY FOR PRESENTATION:")
print("   • Use 'job-postings.xlsx' for Slide 18 in your capstone presentation")
print("   • Create bar chart ordered by job postings (descending)")
print("   • Include key insights in your analysis")
print()
print("🔍 RECOMMENDED ANALYSIS:")
print("   • Compare with Stack Overflow survey results")
print("   • Analyze salary trends by technology")
print("   • Identify emerging technology opportunities")